# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending a question to it and getting it scored for matches in the original questions.

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [1]:
import warnings
warnings.filterwarnings(action="ignore", message="numpy.dtype size changed")

import os
import json
from utilities import write_json_to_file
%load_ext dotenv

## Setup

Below are the various name definitions for the resources needed to setup AKS.

In [2]:
%%writefile --append .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use 
selected_subscription = "YOUR_SUBSCRIPTION"

# Resource group, name and location for AKS cluster.
resource_group = "RESOURCE_GROUP" 
aks_name = "AKS_CLUSTER_NAME"
location = "eastus"

Appending to .env


In [2]:
%%writefile --append .env

# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use 
selected_subscription = "Team Danielle Internal"

# Resource group, name and location for AKS cluster.
resource_group = "fbmlakstestrg" 
aks_name = "fbmlaksclus"
location = "eastus"

Appending to .env


In [3]:
%dotenv
image_name = os.getenv('docker_login') + os.getenv('image_repo')

In [4]:
image_name

'caia/mlaksdep'

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [5]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FPHSBG9T2 to authenticate.
CloudName    IsDefault    Name                                            State    TenantId
-----------  -----------  ----------------------------------------------  -------  ------------------------------------
AzureCloud   False        Boston DS Dev                                   Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Internal Consumption                            Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   True         Team Danielle Internal                          Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Azure Stack Diagnostics CI and Production VaaS  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Core-ES-BLD                                     Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Cosmos_WDG_Core_BnB_100348         

In [5]:
!az account set --subscription "$selected_subscription"

In [6]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [ ]:
!az provider register -n Microsoft.ContainerService

In [ ]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [7]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbmlakstestrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbmlakstestrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster  with 5 nodes in the resource group we created earlier. This step can take ten or more minutes.

In [8]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 5,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_D4_v2",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbmlaksclu-fbmlakstestrg-edf507",
  "fqdn": "fbmlaksclu-fbmlakstestrg-edf507-8c3abe83.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbmlakstestrg/providers/Microsoft.ContainerService/managedClusters/fbmlaksclus",
  "kubernetesVersion": "1.9.9",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [9]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [10]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbmlaksclus" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [11]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-34598344-0   Ready     agent     8m        v1.9.9
aks-nodepool1-34598344-1   Ready     agent     9m        v1.9.9
aks-nodepool1-34598344-2   Ready     agent     9m        v1.9.9
aks-nodepool1-34598344-3   Ready     agent     9m        v1.9.9
aks-nodepool1-34598344-4   Ready     agent     9m        v1.9.9


Let's check the pods on our cluster.

In [12]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-7c677567f6-tjb84             1/1       Running   3          8m
kube-system   heapster-6fbcd8c68f-sbghs               2/2       Running   0          6m
kube-system   kube-dns-v20-784dd5dbd5-jvzh6           3/3       Running   0          8m
kube-system   kube-dns-v20-784dd5dbd5-php9q           3/3       Running   0          8m
kube-system   kube-proxy-4pqrn                        1/1       Running   0          8m
kube-system   kube-proxy-hj5c5                        1/1       Running   0          8m
kube-system   kube-proxy-msrqn                        1/1       Running   0          8m
kube-system   kube-proxy-rrktt                        1/1       Running   0          8m
kube-system   kube-proxy-w8bp6                        1/1       Running   0          8m
kube-system   kube-svc-redirect-46whz                 1/1       Running   0          8m
kube-system   kube-svc-redirect

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the image name and cpu requests and limits for pods. We first start with  deploying 2 pods.

In [13]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-ml"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-ml",
                      "image": image_name,

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-ml"
      }
   }
}

In [14]:
write_json_to_file(app_template, 'az-ml.json')

In [15]:
write_json_to_file(service_temp, 'az-ml.json', mode='a')

Let's check the manifest created.

In [16]:
!cat az-ml.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-ml"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-ml"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "caia/mlaksdep",
                        "name": "azure-ml",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
                

Next, we will use kubectl create command to deploy our application.

In [17]:
!kubectl create -f az-ml.json

deployment.apps/azure-ml created
service/azure-ml created


Let's check if the pod is deployed. It can take several minutes for the deployment to be ready and running.

In [21]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d4d8c754b-dvktm               1/1       Running   0          18m
default       azure-ml-7d4d8c754b-hf97s               1/1       Running   0          18m
kube-system   azureproxy-7c677567f6-tjb84             1/1       Running   3          27m
kube-system   heapster-6fbcd8c68f-sbghs               2/2       Running   0          25m
kube-system   kube-dns-v20-784dd5dbd5-jvzh6           3/3       Running   0          27m
kube-system   kube-dns-v20-784dd5dbd5-php9q           3/3       Running   0          27m
kube-system   kube-proxy-4pqrn                        1/1       Running   0          27m
kube-system   kube-proxy-hj5c5                        1/1       Running   0          27m
kube-system   kube-proxy-msrqn                        1/1       Running   0          27m
kube-system   kube-proxy-rrktt                        1/1       Running   0          27m
kube-system   kube-pr

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [22]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                    SOURCE                                 MESSAGE
29m         29m          1         aks-nodepool1-34598344-0.154901c0026d3a54    Node                                     Normal    Starting                  kubelet, aks-nodepool1-34598344-0      Starting kubelet.
28m         29m          8         aks-nodepool1-34598344-0.154901c0044b32a1    Node                                     Normal    NodeHasSufficientDisk     kubelet, aks-nodepool1-34598344-0      Node aks-nodepool1-34598344-0 status is now: NodeHasSufficientDisk
28m         29m          8         aks-nodepool1-34598344-0.154901c0044b5e61    Node                                     Normal    NodeHasSufficientMemory   kubelet, aks-nodepool1-34598344-0      Node aks-nodepool1-34598344-0 status is now: NodeHasSufficientMemory
28m         29m          7         aks-nodepool1-34598

Check the logs for the first application pod.

In [23]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [24]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-08-08 20:29:19,353 CRIT Supervisor running as root (no user in config file)
2018-08-08 20:29:19,355 INFO supervisord started with pid 1
2018-08-08 20:29:20,357 INFO spawned: 'program_exit' with pid 9
2018-08-08 20:29:20,358 INFO spawned: 'nginx' with pid 10
2018-08-08 20:29:20,360 INFO spawned: 'gunicorn' with pid 11
2018-08-08 20:29:21,392 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
{"stack_info": null, "tags": [], "level": "INFO", "logger": "model_driver", "message": "Model object loading time: 808.79 ms", "timestamp": "2018-08-08T20:29:21.978340Z", "host": "azure-ml-7d4d8c754b-dvktm", "path": "/code/driver.py"}
Initialising
{"msg": " * Running on %s://%s:%d/ %s", "stack_info": null, "tags": [], "level": "INFO", "logger": "werkzeug", "message": " * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)", "timestamp": "2018-08-08T20:29:21.984673Z", "host": "azure-ml-7d4d8c754b-dvktm", "path": "/opt/conda/envs/py3.5/li

In [25]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   2         2         2            2           19m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [26]:
!kubectl get service azure-ml

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP      PORT(S)        AGE
azure-ml   LoadBalancer   10.0.9.28    40.117.112.173   80:32091/TCP   19m


# Scaling

In this part, we scale the number of pods to make sure we fully utilize the AKS cluster.

In [27]:
!kubectl scale --current-replicas=2 --replicas=35 deployment/azure-ml

deployment.extensions/azure-ml scaled


It can take a couple of minutes for all replicas to be running.

In [30]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d4d8c754b-28cvf               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-2zpnl               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-464d4               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-4fkrp               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-66trz               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-78jtw               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-7nmvm               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-7t5jj               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-99w27               1/1       Running   0          3m
default       azure-ml-7d4d8c754b-blscw               1/1       Running   0          3m
default       azure-ml-7d4d8c75

In [31]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   35        35        35           35          22m


Next, we will [test our web application deployed on AKS](06_Test_WebApp.ipynb).

Once, you are done with all the notebooks of the tutorial, you can use the instructions in the [last notebook](09_Tear_Down.ipynb) to tear down the cluster.